## Library

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import emoji
import json
import csv

message, from, created_time, comments.limit(0).summary(1), reactions.limit(0).summary(1), reactions.type(LIKE).limit(0).summary(1).as(like), reactions.type(LOVE).limit(0).summary(1).as(love), reactions.type(HAHA).limit(0).summary(1).as(haha), reactions.type(WOW).limit(0).summary(1).as(wow), reactions.type(SAD).limit(0).summary(1).as(sad), reactions.type(ANGRY).limit(0).summary(1).as(angry)

In [24]:
# # Drop not-using columns
# not_using_col = ["id", "parent_id", "level", "object_id", "query_status", "query_time", "query_type",
#                 "paging.cursors.before", "paging.cursors.after", "paging.next", "reactions.summary.viewer_reaction",
#                 "sad.summary.viewer_reaction", "angry.data", "angry.summary.viewer_reaction", "updated_time", "error.message",
#                 "reactions.data", "id.1", "like.summary.total_count", "like.summary.viewer_reaction", "sad.data",
#                 "error.type", "error.code", "error.fbtrace_id", "from.name", "from.id", "like.data", "object_key", 
#                 "object_type", "reactions.summary.total_count", "sad.summary.total_count", "angry.summary.total_count"]
# df.drop(columns = not_using_col, inplace=True)

## 1. Clean df

In [103]:
#*********************************
Bank_name = 'HSBC'
#*********************************

Filename = f'FB_{Bank_name}_2022_v2.csv'
New_Filename = f'FBClean_{Bank_name}_v2.csv'

df = pd.read_csv(f'Collected_data/{Filename}', header=0)

In [104]:
# Drop non-data and NaN messages
drop_index = df[df.object_type != "data"].index
df.drop(axis = 0, index = drop_index, inplace = True)

drop_index = df[df.message.isna()].index
df.drop(axis = 0, index = drop_index, inplace = True)

# Categorization of reactions
df['pos_reaction'] = df['reactions.summary.total_count'] - df['sad.summary.total_count'] - df['angry.summary.total_count']
df['neg_reaction'] = df['sad.summary.total_count'] + df['angry.summary.total_count']

df = df.rename(columns = {'shares.count' : 'shares', 'comments.summary.total_count' : 'comments'})

In [105]:
df.columns

Index(['path', 'id', 'parent_id', 'level', 'object_id', 'object_type',
       'object_key', 'query_status', 'query_time', 'query_type', 'message',
       'created_time', 'updated_time', 'error.message', 'from.name', 'from.id',
       'comments.data', 'comments.summary.order', 'comments',
       'comments.summary.can_comment', 'reactions.data',
       'reactions.summary.total_count', 'reactions.summary.viewer_reaction',
       'like.data', 'like.summary.total_count', 'like.summary.viewer_reaction',
       'love.data', 'love.summary.total_count', 'love.summary.viewer_reaction',
       'haha.data', 'haha.summary.total_count', 'haha.summary.viewer_reaction',
       'wow.data', 'wow.summary.total_count', 'wow.summary.viewer_reaction',
       'sad.data', 'sad.summary.total_count', 'sad.summary.viewer_reaction',
       'angry.data', 'angry.summary.total_count',
       'angry.summary.viewer_reaction', 'id.1', 'summary.order',
       'summary.total_count', 'summary.can_comment', 'comment_count'

In [106]:
# Replace NaN with zero
df["shares"] = df["shares"].replace(np.nan,"0")
df["comments"] = df["comments"].replace(np.nan,"0")
df["pos_reaction"] = df["pos_reaction"].replace(np.nan,"0")
df["neg_reaction"] = df["neg_reaction"].replace(np.nan,"0")

# Change of column type to int
df["shares"] = df["shares"].apply(lambda x: int(x))
df["comments"] = df["comments"].apply(lambda x: int(x))
df["pos_reaction"] = df["pos_reaction"].apply(lambda x: int(x))
df["neg_reaction"] = df["neg_reaction"].apply(lambda x: int(x))

In [107]:
# Datetime
df["created_time"] = pd.to_datetime(df["created_time"])
df["date"] = df["created_time"].apply(lambda x: x.date())
df["date"] = pd.to_datetime(df["date"])

In [108]:
# Add bank name
df["bank"] = Bank_name

In [109]:
df = df.reindex(columns = ['bank', 'date','message', 'pos_reaction', 'neg_reaction', 'shares', 'comments'])

# Drop duplicate rows
df.drop_duplicates(inplace=True)

# Reset index
df.reset_index(drop=True, inplace=True)

In [110]:
df.head(5)

,bank,date,message,pos_reaction,neg_reaction,shares,comments
0,HSBC,2022-09-30,【滙豐 Mobile Banking X 數碼科技 X 音樂🎵】 邊個話Banking 同...,29,0,0,4
1,HSBC,2022-09-30,【免佣免平台費 嬴在投資起跑線💨｜滙豐「調教少年股神」】 開Sem後又要chur grou...,15,1,0,2
2,HSBC,2022-09-29,【究竟乜嘢係元宇宙？web 3.0? DAO? VR?】 這些詞語和現實世界嘅我如何扯上關係...,13,1,0,1
3,HSBC,2022-09-29,【未響警號🚨就諗住遲啲先？無論身體健康定財政健康都唔好再拖喇！一齊為咗財政健康，戰勝拖延症！...,35,3,1,1
4,HSBC,2022-09-28,#睇片贏獎品【國泰航空／滙豐香港國際七人欖球賽有獎問答遊戲，贏取現場觀賽機會📣】 萬眾曯目嘅...,801,1,300,846


In [111]:
# Double-checking
df[df["neg_reaction"] < 0]

,bank,date,message,pos_reaction,neg_reaction,shares,comments


In [112]:
# Double-checking
df[df["pos_reaction"] < 0]

,bank,date,message,pos_reaction,neg_reaction,shares,comments


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   bank          210 non-null    object        
 1   date          210 non-null    datetime64[ns]
 2   message       210 non-null    object        
 3   pos_reaction  210 non-null    int64         
 4   neg_reaction  210 non-null    int64         
 5   shares        210 non-null    int64         
 6   comments      210 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 11.6+ KB


In [114]:
df.to_csv(f'Collected_data/{New_Filename}', encoding='utf-8-sig', index=False)

## 2. Concat df

In [115]:
df_1 = pd.read_csv('Collected_data/FBClean_ZA_v2.csv', header=0)
df_2 = pd.read_csv('Collected_data/FBClean_SC_v2.csv', header=0)
df_3 = pd.read_csv('Collected_data/FBClean_ICBC_v2.csv', header=0)
df_4 = pd.read_csv('Collected_data/FBClean_HSBC_v2.csv', header=0)
df_5 = pd.read_csv('Collected_data/FBClean_DBS_v2.csv', header=0)
df_6 = pd.read_csv('Collected_data/FBClean_BEA_v2.csv', header=0)

In [116]:
# Concat all df
frames = [df_1, df_2, df_3, df_4, df_5, df_6]  
df_master = pd.concat(frames, ignore_index=True)

In [117]:
display(df_master)

,bank,date,message,pos_reaction,neg_reaction,shares,comments
0,ZA,2022-09-26,"【突發：中巨獎有秘訣？有人用 $60 就贏到 $1,000,000?!】 呢個人就係新一位...",23,5,0,18
1,ZA,2022-09-23,【又食又拎基本啦！🍽 食勻港九再拎足 11% 回贈！😋】 TGIF! 常識題：放假食乜好...,20,2,4,3
2,ZA,2022-09-17,【SHAKE SHACK x ZA Bank 賞你免費漢堡】😋 過咗中秋，和煦嘅陽光下終於...,1102,0,618,1080
3,ZA,2022-09-16,【百萬勁抽限定！L 先生連環幸運事件】 23歲就有一百萬係咩概念？小編答你唔到，不過可以幫...,33,7,0,22
4,ZA,2022-09-10,【究竟邊個中咗一百萬 🤑⁉️】 數字有分先後！今期「百萬勁抽」獨贏號碼喺… 🥁 🥁 🥁 ...,51,8,3,36
...,...,...,...,...,...,...,...
718,BEA,2022-04-06,【第2季投資策略🎯】 上季資產市場一如舊年年底所料出現大幅波動。上季同大家分享咗3大投資主...,10,0,1,1
719,BEA,2022-04-04,【BEA Mall每月大抽獎🎁 春日簽賬贏取HK$500電子購物禮券】 踏入春季天氣濕笠笠...,292,3,6,3
720,BEA,2022-04-01,【電子錢包消費🌟享高達HK$80獎賞】 呢排抗疫時期，做咩都要打醒十二分精神！出街搭車同...,122,2,9,5
721,BEA,2022-04-01,【BEA Flash 開戶一觸即發】 劍擊講求靈活，反應要敏捷，動作要快先可以一擊即中。 ...,298,3,1,16


In [118]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723 entries, 0 to 722
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   bank          723 non-null    object
 1   date          723 non-null    object
 2   message       723 non-null    object
 3   pos_reaction  723 non-null    int64 
 4   neg_reaction  723 non-null    int64 
 5   shares        723 non-null    int64 
 6   comments      723 non-null    int64 
dtypes: int64(4), object(3)
memory usage: 39.7+ KB


In [120]:
# df_master.to_csv('Collected_data/FB_Banks_MasterData_v2.csv', encoding='utf-8-sig', index=False)

## End